In [1]:
import hotspot
import scanpy as sc
import muon as mu
import numpy as np
import mplscience
import h5py
import os
import pandas as pd
from scipy.sparse import csc_matrix

/home/am3019/venvs/hotspot/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dts_name = "immune"
app_path = f"/mnt/h/Bioinf_data/Pipelines/starlng_app_{dts_name}"
objects_path = os.path.join(app_path, "objects")

In [3]:
expr_path = os.path.join(objects_path, "expression.h5")


In [4]:
adata = sc.read_h5ad("/mnt/h/Bioinf_data/Pipelines/ClustAssess-paper/immune.h5ad")

In [5]:
adata.obs["total_counts"] = adata.X.sum(axis=1)

In [6]:
adata.layers["counts"] = adata.X.copy()
sc.pp.filter_genes(adata, min_cells = 10)
sc.pp.normalize_total(adata)
sc.pp.log1p(adata)
adata.layers["log_normalized"] = adata.layers["counts"]
sc.pp.scale(adata)
sc.tl.pca(adata)

In [7]:
# convert sparse to dense
adata.layers["counts"] = adata.layers["counts"].todense()

In [8]:
hs = hotspot.Hotspot(
    adata,
    layer_key = "counts",
    model = "danb",
    latent_obsm_key = "X_pca",
    umi_counts_obs_key = "total_counts"
)

In [9]:
np.random.seed(202408)
seeds = np.random.randint(1, 1000, 30)
seeds

array([855, 330, 306, 828, 574, 713, 181, 520, 207, 244, 290, 571, 360,
       782,  85, 538, 528, 733, 826, 829, 161, 476, 226,  85, 287, 800,
       158, 436, 936,  33])

In [10]:
modules_dict = dict()
for seed in seeds:
    np.random.seed(seed)
    hs.create_knn_graph(weighted_graph = False, n_neighbors = 30)
    hs_results = hs.compute_autocorrelations(jobs = 30)

    hs_genes = hs_results.loc[hs_results.FDR < 0.01].sort_values('Z', ascending=False).index
    lcz = hs.compute_local_correlations(hs_genes, jobs = 30)

    modules = hs.create_modules(
        min_gene_threshold = 10,
        fdr_threshold = 0.05,
        core_only = True
    )
    modules_dict[str(seed)] = modules

100%|██████████| 22366/22366 [00:12<00:00, 1835.81it/s]


Computing pair-wise local correlation on 14250 features...


  4%|▎         | 3783587/101524125 [05:47<2:29:30, 10896.30it/s]


KeyboardInterrupt: 